In [ ]:
!pip install -U --user numpy

In [ ]:
!pip install wget

In [1]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import urllib
import wget

MAX_LEN = 512
EPOCHS = 3
VERBOSE = 2
BATCH_SIZE = 16

In [2]:
DATA_OUT_PATH = './data_out/KOR'

In [3]:
def plot_graphs(history, string, string_1, string_2):
    # loss 
    plt.plot(history.history[string])
    plt.plot(history.history[string_1])
    plt.plot(history.history[string_2])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, string_1, string_2])
    plt.show()

In [4]:
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)
np.random.seed(SEED_NUM)

In [5]:
# Save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", lowercase=False)
save_path = "bert-base-multilingual-cased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert-base-multilingual-cased/vocab.txt", lowercase=False)

In [6]:
train_data_url = "https://korquad.github.io/dataset/KorQuAD_v1.0_train.json"
train_path = keras.utils.get_file("train.json", train_data_url)
eval_data_url = "https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json"
eval_path = keras.utils.get_file("eval.json", eval_data_url)

In [ ]:
wget.download('https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json', out='./bert-base-multilingual-cased/')

In [ ]:
os.rename('./bert-base-multilingual-cased/bert-base-multilingual-cased-config.json', './bert-base-multilingual-cased/config.json')

In [ ]:
wget.download('https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-tf_model.h5', out='./bert-base-multilingual-cased/')

In [ ]:
os.rename('./bert-base-multilingual-cased/bert-base-multilingual-cased-tf_model.h5', './bert-base-multilingual-cased/tf_model.h5')

In [7]:
class SquadExample:
    def __init__(self, question, context, start_char_idx, answer_text):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        # Find end character index of answer in context
        end_char_idx = start_char_idx + len(answer)
        if end_char_idx >= len(context):
            self.skip = True
            return

        # Mark the character indexes in context that are in answer
        is_char_in_ans = [0] * len(context)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1

        # Tokenize context
        tokenized_context = tokenizer.encode(context)

        # Find tokens that were created from answer characters
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_context.offsets):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)

        if len(ans_token_idx) == 0:
            self.skip = True
            return

        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = MAX_LEN - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets


def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                answer_text = qa["answers"][0]["text"]
                start_char_idx = qa["answers"][0]["answer_start"]
                squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text
                )
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
    return squad_examples


def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y


In [8]:
with open(train_path) as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)


train_squad_examples = create_squad_examples(raw_train_data)
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

eval_squad_examples = create_squad_examples(raw_eval_data)
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")

60407 training points created.
5774 evaluation points created.


In [9]:
idx=22
print(x_train[0].shape)
# print(x_train[0][0])
text = tokenizer.decode(x_train[0][idx])
print(text)
start = y_train[0][idx]
end = y_train[1][idx]
print(start,end)

(56282, 512)
이 작품은 라단조, Sehr gehalten ( 아주 신중하게 ), 4 / 4박자의 부드러운 서주로 서주로 시작되는데, 여기에는 주요 주제, 동기의 대부분이 암시, 예고되어 있다. 첫 부분의 저음 주제는 주요 주제 ( 고뇌와 갈망 동기, 청춘의 사랑 동기 ) 를 암시하고 있으며, 제1바이올린으로 더욱 명확하게 나타난다. 또한 그것을 이어받는 동기도 중요한 역할을 한다. 여기에 새로운 소재가 더해진 뒤에 새로운 주제도 연주된다. 주요부는 Sehr bewegt ( 아주 격동적으로 ), 2 / 2박자의 자유로운 소나타 형식으로 매우 드라마틱한 구상과 유기적인 구성을 하고 있다. 여기에는 지금까지의 주제나 소재 외에도 오보에에 의한 선율과 제2주제를 떠올리게 하는 부차적인 주제가 더해지는데, 중간부에서는 약보3이 중심이 되고 제2주제는 축소된 재현부에서 D장조로 재현된다. 마지막에는 주요 주제를 회상하면서 조용히 마친다. 이 곡의 주요 주제는?
71 85


In [10]:
class TFBERTQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBERTQuestionAnswering, self).__init__()
        
        self.encoder = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.start_logit = tf.keras.layers.Dense(num_class, name="start_logit", use_bias=False)
        self.end_logit = tf.keras.layers.Dense(num_class, name="end_logit", use_bias=False)
        self.flatten = tf.keras.layers.Flatten() 
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)
        
    def call(self, inputs):
        input_ids, token_type_ids, attention_mask = inputs
        embedding = self.encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
        start_logits = self.start_logit(embedding)
        start_logits = self.flatten(start_logits)
        
        end_logits = self.end_logit(embedding)
        end_logits = self.flatten(end_logits)
        
        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)
    
        return start_probs, end_probs

In [11]:
korquad_model = TFBERTQuestionAnswering(model_name='./bert-base-multilingual-cased/',dir_path='bert_ckpt', num_class=1)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)

Some layers from the model checkpoint at ./bert-base-multilingual-cased/ were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at ./bert-base-multilingual-cased/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [29]:
def normalized_answer(s):    
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text) 
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)   
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)      
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

In [30]:
class ExactMatch(keras.callbacks.Callback):
    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]

            normalized_pred_ans = normalized_answer(pred_ans)
            normalized_true_ans = normalized_answer(squad_eg.answer_text)
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")

In [13]:
exact_match_callback = ExactMatch(x_eval, y_eval)

In [14]:
korquad_model.compile(optimizer=optimizer, loss=[loss, loss])

In [15]:
model_name = "tf2_bert_korquad"

checkpoint_path = os.path.join(DATA_OUT_PATH, model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, verbose=1, save_best_only=True, save_weights_only=True)

./data_out/KOR\tf2_bert_korquad -- Folder already exists 



In [16]:
history = korquad_model.fit(
    x_train,
    y_train,
    epochs=EPOCHS,  # For demonstration, 3 epochs are recommended
    verbose=VERBOSE,
    batch_size=BATCH_SIZE,
    callbacks=[exact_match_callback, cp_callback]
)

Epoch 1/3


KeyboardInterrupt: 

In [ ]:
print(history.history)

In [ ]:
plot_graphs(history, 'loss', 'output_1_loss', 'output_2_loss')

In [ ]:
korquad_model.save_weights('data_out/KOR/tf2_bert_korquad/weights_KorQuAD.h5')

In [17]:
korquad_model.load_weights('data_out/KOR/tf2_bert_korquad/weights_KorQuAD.h5')

In [ ]:
korquad_model.loss

In [ ]:
plot_graphs(korquad_model.history,'loss', 'output_1_loss', 'output_2_loss')

### 모델 predict 구현

In [ ]:
korquad_model.load_weights('data_out/KOR/tf2_bert_korquad/weights_KorQuAD.h5')

#### 지문 찾아보기

In [18]:
print(tokenizer.decode(x_eval[0][1]))

1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다. 1989년 6월 30일 평양축전에 대표로 파견 된 인물은?


In [19]:
print(x_eval[1][1])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [20]:
tokenized_context= tokenizer.encode("1989년 6월 30일 평양축전에 대표로 파견 된 인물은?")

In [27]:
print(tokenized_context.ids)

[101, 76485, 17253, 40636, 9926, 37114, 70122, 68767, 9069, 37824, 11261, 9901, 118634, 9099, 9640, 29364, 10892, 136, 102]


In [21]:
context_token_to_char = tokenized_context.offsets
print(context_token_to_char)

[(0, 0), (0, 5), (6, 8), (9, 12), (13, 14), (14, 15), (15, 16), (16, 18), (19, 20), (20, 21), (21, 22), (23, 24), (24, 25), (26, 27), (28, 29), (29, 30), (30, 31), (31, 32), (0, 0)]


In [22]:
for i in range(19):
    print(tokenizer.decode([tokenized_context.ids[i]]), end=" ")

 1989년 6월 30일 평 ##양 ##축 ##전에 대 ##표 ##로 파 ##견 된 인 ##물 ##은 ?  

In [41]:
class SquadExample_pred:
    def __init__(self, question, context):#, start_char_idx, answer_text):
        self.question = question
        self.context = context
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        #answer_text = self.answer_text
        #start_char_idx = self.start_char_idx

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())

        # Tokenize context
        tokenized_context = tokenizer.encode(context)
#         print("tokenized_context.ids=", tokenized_context.ids)
#         print("tokenized_context.ids.len=", len(tokenized_context.ids))

        # Tokenize question
        tokenized_question = tokenizer.encode(question)
#         print("tokenized_question.ids=", tokenized_question.ids)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        
        attention_mask = [1] * len(input_ids)
        

        # Pad and create attention masks.
        # Skip if truncation is needed
        
        padding_length = MAX_LEN - len(input_ids)
        print("padding_length=",padding_length)
        
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return
        
#         print("input_ids=", input_ids)
#         print("token_type_ids=", token_type_ids)
#         print("attention_mask=", attention_mask)

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.context_token_to_char = tokenized_context.offsets
#         print(self.context_token_to_char)
        
    def get_input_target(self):
        dataset_dict = {
            "input_ids": [],
            "token_type_ids": [],
            "attention_mask": [],
        }
        if self.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(self, key))
        for key in dataset_dict:
            dataset_dict[key] = np.array(dataset_dict[key])

        x = [
            dataset_dict["input_ids"],
            dataset_dict["token_type_ids"],
            dataset_dict["attention_mask"],
        ]
#         print(x)
        return x

def create_squad_examples_from_arg(question, context):#, start_char_idx, answer_text):
    squad_eg = SquadExample_pred(
        question, context#, start_char_idx, answer_text
    )
    squad_eg.preprocess()
    return squad_eg


def predict_test(model, pred_raw):
    x_pred = pred_raw.get_input_target()
    pred_start, pred_end = model.predict(x_pred)
    
    pred_start_offset_index = np.argmax(pred_start)
    pred_end_offset_index = np.argmax(pred_end)
#     print(pred_start_offset_index)
#     print(pred_end_offset_index)
    pred_start_offset = pred_raw.context_token_to_char[pred_start_offset_index]
#     print(pred_raw.input_ids)
#     print(pred_start_offset)
    pred_end_offset = pred_raw.context_token_to_char[pred_end_offset_index]
#     print(pred_end_offset)
    answer = pred_context[pred_start_offset[0]:pred_end_offset[1]]
#     print(pred_context)
#     print(answer)
    
    normalized_pred_ans = normalized_answer(answer)
    
    return normalized_pred_ans


pred_context = "해리 케인이 토트넘 홋스퍼 퇴단을 원한다는 보도가 나왔다. 영국 매체 익스프레스는 21일(한국 시간) “케인이 토트넘을 떠나고 싶어 한다. 하지만 다니엘 레비 회장의 거센 반대에 직면할 것”이라고 보도했다. 매체는 유럽 이적시장 전문가인 데이비드 온스테인 기자의 인터뷰 내용을 덧붙였다. 온스테인 기자는 21일 영국 스카이 스포츠에 출연해 “난 축구계에 있는 많은 사람과 얘길 나눈다. 케인이 토트넘을 떠나고 싶어 한단 생각은 일치한다”고 말했다. 케인은 토트넘 유스 출신이다. 밀월 FC, 레스터 시티 등 4번의 임대 생활을 제하면 줄곧 토트넘에서만 뛰었다. 케인은 2014/15시즌부터 주전 공격수로 자리 잡았다. 이후 2차례나 프리미어리그 득점왕을 거머쥐었다. 올 시즌에도 활약은 이어지고 있다. 리그 27경기에 나서 17골 13도움을 기록하며 득점 공동 선두, 도움 1위를 질주 중이다. 개인 성적은 훌륭하지만, 한 가지 아쉬운 게 있다. 우승 트로피가 없단 것. 사실 이번 시즌 토트넘은 리그 제패에 대한 기대가 있었다. 시즌 초 리그 11경기 무패 행진(7승 4무)으로 상위권을 유지했다. 하지만 일관성이 부족했다. 이내 고꾸라졌고 현재는 6위에 있다. 현실적인 목표는 유럽축구연맹(UEFA) 챔피언스리그 티켓을 거머쥐는 것이다. 우승 가능성이 있었던 FA컵, UEFA 유로파리그도 떨어졌다. 남은 대회는 카라바오컵이다. 하지만 이마저도 맨체스터 시티와 결승전을 치른다. 우승을 장담할 수 없다. 결국 우승 트로피 때문에 토트넘을 떠나고 싶어 하는 것으로 보인다.  "
pred_question = "반대하는 사람?"
# pred_question = "유럽 이적시장 전문가?"

pred_data = create_squad_examples_from_arg(pred_question, pred_context)
pred_answer = predict_test(korquad_model, pred_data)
print(pred_answer)

padding_length= 40
다니엘 레비


In [42]:
pred_context = "1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다."
# pred_question = "1989년 6월 30일 평양축전에 대표로 파견 된 인물은?"
pred_question = "임종석의 사전구속영장을 발부받은 날짜는?"
pred_data = create_squad_examples_from_arg(pred_question, pred_context)
pred_answer = predict_test(korquad_model, pred_data)
print(pred_answer)

padding_length= 202
1989년 3월 12일


In [47]:
pred_context = '''지난 23일 저녁 열린 텐센트의 작년 4분기 실적 발표 콘퍼런스 콜에서는 "중국 당국의 눈에 너무 비대한 것으로 비치지 않기 위해 사업을 분할할 가능성이 있냐"는 민감한 질문이 나왔다.
그러자 그때까지 질의에 답하던 텐센트의 전략총괄책임자 제임스 미첼은 바로 마이크를 텐센트 창업자 겸 최고경영자(CEO) 마화텅에게 넘겼는데, 마화텅이 입을 열기도 전에 텐센트 그룹 회장인 마틴 라우가 이같이 잘라 말했다.
홍콩 사우스차이나모닝포스트(CMP)는 25일 "이런 민감한 질문의 배경에는 중국 정부가 자국 인터넷 분야의 절반을 장악한 것으로 알려진 24년 된 소셜미디어·비디오게임 거인인 텐센트를 쪼개고 싶어한다는 이야기에 대한 투자자들의 커지는 우려가 자리한다"고 보도했다.
앞서 이달 로이터 통신에는 "텐센트를 쪼개는 것은 늘어만 가는 당국의 규제 장벽을 넘어설 수 있는 과격하지만 간단한 방법"이라는 기고문이 실렸다.
SCMP는 텐센트의 기업분할이 진행 중이라는 어떠한 증거도 없지만 중국 당국이 '자본의 비이성적인 확장'을 제한하려는 가운데 텐센트의 비대한 몸집은 당국의 골칫거리이며, 당국의 규제가 계속해서 텐센트의 앞날에 그림자를 드리울 것이라는 전망이 나온다고 전했다.'''
# pred_question = "텐센트의 창업자는?"
pred_question = "마화텅에게 마이크를 넘긴 사람은?"

pred_data = create_squad_examples_from_arg(pred_question, pred_context)
pred_answer = predict_test(korquad_model, pred_data)
print(pred_answer)

padding_length= 126
제임스 미첼


In [48]:
pred_context = '''25일(현지 시각) 파이낸셜타임스(FT)는 세계적인 석유 트레이더들의 회의 인터뷰를 통해 이같이 전했다.
보도에 따르면 이 부분의 가장 유명한 헤지펀드 매니저 중 한 명인 피에르 안두랑은 러시아의 우크라이나 침공 이후 유럽으로의 러시아 석유 공급이 사라져 에너지 시장이 재편될 것이라고 말했다.
그는 “몇달 안에 정상으로 돌아가지 않을 것”이라면서 “유럽에서 러시아의 공급을 영원히 잃을 것 같다”고 언급했다. 그는 올해 원유 가격이 현재보다 두배 높은 배럴당 250달러에 달할 수도 있다고 덧붙였다.
이 회의에서 연설한 다른 석유 시장 베테랑들은 우크라이나와의 휴전이 합의되더라도 러시아산 원유와 경유 등 정제제품은 조만간 유럽 시장에 복귀하지 않을 것이라고 입을 모았다.
분석가들은 하루에 300배럴의 러시아 원유가 시장에서 손실될 수 있다고 추정했다.'''

pred_question = "원유 가격이 250달러가 될것이라고 말한 사람은?"

pred_data = create_squad_examples_from_arg(pred_question, pred_context)
pred_answer = predict_test(korquad_model, pred_data)
print(pred_answer)

padding_length= 238
피에르 안두랑
